In [179]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import probplot
from pandas.plotting import scatter_matrix
#import missingno as msno
from IPython.display import Markdown, display
import statsmodels.api as sm
from statsmodels.tools import add_constant as add_constant
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
#from imblearn.over_sampling import SMOTE
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from datetime import date
from datetime import datetime

In [180]:
#df = pd.read_csv('https://raw.githubusercontent.com/MarlaGoodman/CMS-Project/main/flights.csv')
#df = pd.read_csv('https://raw.githubusercontent.com/MarlaGoodman/CMS-Project/main/flights_2.csv')
df = pd.read_csv('https://raw.githubusercontent.com/MarlaGoodman/CMS-Project/main/flights_model.csv')

In [181]:
df['response_var'] = np.where(df['dep_delay']> 0, 1, 0)

In [182]:
#df['time_hour']=pd.to_datetime(df['time_hour'])

In [183]:
#s = pd.to_datetime(df['time_hour'])
#day_df = s.groupby(s.dt.floor('d')).size().reset_index(name='count')
#day_df.head()

In [184]:
#df['time_hour_day']=df['time_hour'].dt.floor('d')

In [185]:
#def flight_count_by_day(row):
#    for i in range(len(df.month)):
#        j = np.where(day_df['time_hour']==df['time_hour_day'][i])
#        count = day_df['count'][j[0][0]]
#        #print (i)
#        return count

In [186]:
#j= np.where(day_df['time_hour']==df['time_hour_day'][100150])
#j[0][0]
#count = day_df['count'][j[0][0]]
#count

In [187]:
#df['flight_count_by_day'] = df.apply(flight_count_by_day, axis=1)

In [188]:
#df_flight_count_by_day = df['flight_count_by_day']
#df_flight_count_by_day
#print(type(df_flight_count_by_day))
#pd.DataFrame(df_flight_count_by_day).groupby(df_flight_count_by_day)

#grouped = df_flight_count_by_day.groupby(lambda x: df_flight_count_by_day[x])
#pd.DataFrame(grouped)
#df_flight_count_by_day.groupby(df_flight_count_by_day).size()


In [189]:
#df.groupby(['carrier']).count()['month']

In [190]:
#def weekday(row):
#    for i in range(len(df.month)):
#        return( date(2017, df['month'][i], df['day'][i]).isoweekday())

In [191]:
#df['weekday'] = df.apply(weekday, axis=1)

In [192]:
#def weekend(row):
#    for i in range(len(df.month)):
#        if(df['weekday'][i]==5):
#            return 1
#        elif(df['weekday'][i]==6):
#            return 1
#        elif (df['weekday'][i]==7):
#            return 1
#        else:
#            return 0

In [193]:
#df['weekend_bool'] = df.apply(weekend, axis=1)

In [194]:
dummy_month = pd.get_dummies(df['month']).rename(columns=lambda x: 'Month_' + str(x))
dummy_hour = pd.get_dummies(df['hour']).rename(columns=lambda x: 'Hour_' + str(x))
#dummy.head()

In [195]:
df = pd.concat([df,dummy_month], axis=1)
df = pd.concat([df,dummy_hour], axis=1)

df.head()

,dep_delay,month,hour,count_of_flight_by_day_carrier_origin,new_snow_bool,rain_bool,response_var,Month_1,Month_2,Month_3,...,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23
0,193.0,10,21,145,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,-6.0,10,5,145,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-1.0,10,5,25,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,-4.0,10,5,24,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,-8.0,10,6,8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [196]:
X = df['count_of_flight_by_day_carrier_origin']

In [197]:
X_manual_scaled = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))

In [198]:
X_manual_scaled

0         0.813559
1         0.813559
2         0.135593
3         0.129944
4         0.039548
            ...   
303743    0.112994
303744    0.593220
303745    0.593220
303746    0.621469
303747    0.621469
Name: count_of_flight_by_day_carrier_origin, Length: 303748, dtype: float64

In [199]:
del df['count_of_flight_by_day_carrier_origin']

In [200]:
df = pd.concat([df,X_manual_scaled], axis=1)

In [201]:
#df = df.drop(df.columns[[17]], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303748 entries, 0 to 303747
Data columns (total 39 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   dep_delay                              295934 non-null  float64
 1   month                                  303748 non-null  int64  
 2   hour                                   303748 non-null  int64  
 3   new_snow_bool                          303748 non-null  int64  
 4   rain_bool                              303748 non-null  int64  
 5   response_var                           303748 non-null  int32  
 6   Month_1                                303748 non-null  uint8  
 7   Month_2                                303748 non-null  uint8  
 8   Month_3                                303748 non-null  uint8  
 9   Month_4                                303748 non-null  uint8  
 10  Month_5                                303748 non-null  

In [202]:
predictors = []

In [203]:
#predictors.append('hour')

In [204]:
predictors.append('count_of_flight_by_day_carrier_origin')

In [205]:
#predictors.append('flight_count_by_day')

In [206]:
dummy_list = list(dummy_month.columns)
dummy_list.extend(list(dummy_hour.columns))

In [207]:
predictors.extend(dummy_list)

In [208]:
predictors.remove('Hour_1')

In [209]:
predictors.append('rain_bool')

In [210]:
predictors.append('new_snow_bool')

In [211]:
predictors

['count_of_flight_by_day_carrier_origin',
 'Month_1',
 'Month_2',
 'Month_3',
 'Month_4',
 'Month_5',
 'Month_6',
 'Month_7',
 'Month_8',
 'Month_9',
 'Month_10',
 'Month_11',
 'Month_12',
 'Hour_5',
 'Hour_6',
 'Hour_7',
 'Hour_8',
 'Hour_9',
 'Hour_10',
 'Hour_11',
 'Hour_12',
 'Hour_13',
 'Hour_14',
 'Hour_15',
 'Hour_16',
 'Hour_17',
 'Hour_18',
 'Hour_19',
 'Hour_20',
 'Hour_21',
 'Hour_22',
 'Hour_23',
 'rain_bool',
 'new_snow_bool']

In [212]:
X = df[predictors]
y = df['response_var']

#creating train and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

#Fitting Logistic Regression to the Training set with the first set of predictors.
classifier1 = LogisticRegression()
classifier1.fit(X_train, y_train)

LogisticRegression()

In [213]:


#examine the model coefficients for the explanatory variables
print(predictors)
print(classifier1.coef_)

# Cross validation.
accuracy_list = []
recall_list = []
auc_list = []
f1_list = []
precision_list = []

accuracy = cross_val_score(classifier1, X_train, y_train, scoring = "accuracy", cv = 10)
recall = cross_val_score(classifier1, X_train, y_train, scoring = "recall", cv = 10)
auc = cross_val_score(classifier1, X_train, y_train, scoring = "roc_auc", cv = 10)
f1 = cross_val_score(classifier1, X_train, y_train, scoring = "f1", cv = 10)
precision = cross_val_score(classifier1, X_train, y_train, scoring = "precision", cv = 10)

accuracy_list.append(accuracy.mean())
recall_list.append(recall.mean())
auc_list.append(auc.mean())
f1_list.append(f1.mean())
precision_list.append(precision.mean())

['count_of_flight_by_day_carrier_origin', 'Month_1', 'Month_2', 'Month_3', 'Month_4', 'Month_5', 'Month_6', 'Month_7', 'Month_8', 'Month_9', 'Month_10', 'Month_11', 'Month_12', 'Hour_5', 'Hour_6', 'Hour_7', 'Hour_8', 'Hour_9', 'Hour_10', 'Hour_11', 'Hour_12', 'Hour_13', 'Hour_14', 'Hour_15', 'Hour_16', 'Hour_17', 'Hour_18', 'Hour_19', 'Hour_20', 'Hour_21', 'Hour_22', 'Hour_23', 'rain_bool', 'new_snow_bool']
[[ 0.15010136  0.07646878 -0.15120456  0.11641097  0.08197907  0.20998885
   0.19339827  0.16238915 -0.01676285 -0.45005143 -0.33968132 -0.52952467
  -0.00920521 -1.20506901 -1.07336366 -0.91862597 -0.66569663 -0.45571934
  -0.43598836 -0.36506463 -0.33529319 -0.10938253  0.10263517  0.27597926
   0.34308646  0.48216158  0.55364146  0.65716012  0.71198064  0.7385915
   0.56564377  0.47511003  0.26814074  0.41633288]]


In [214]:
accuracy_list

[0.6658079461685921]

In [215]:
recall_list

[0.3419335969196443]

In [216]:
auc_list

[0.6766637274068257]

In [217]:
f1_list

[0.4260171745958797]

In [218]:
precision_list

[0.5649661168856911]

In [219]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.609811
         Iterations 6
                                   Results: Logit
Model:                     Logit                  Pseudo R-squared:       0.069      
Dependent Variable:        response_var           AIC:                    370525.6119
Date:                      2021-04-21 23:32       BIC:                    370886.8263
No. Observations:          303748                 Log-Likelihood:         -1.8523e+05
Df Model:                  33                     LL-Null:                -1.9894e+05
Df Residuals:              303714                 LLR p-value:            0.0000     
Converged:                 1.0000                 Scale:                  1.0000     
No. Iterations:            6.0000                                                    
-------------------------------------------------------------------------------------
                                       Coef.  Std.Err.    z    P>|z|   [0.

In [220]:
from sklearn.metrics import confusion_matrix
y_pred = classifier1.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[32990,  5725],
       [14460,  7575]], dtype=int64)

In [221]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

#defining the scorer
f1_scorer = metrics.make_scorer(metrics.f1_score)
parameters = {'max_depth':np.arange(5,10),'class_weight':['balanced','none']}

In [222]:
# perform grid search to find best hyperparameters
random_f = RandomForestClassifier()
grid = GridSearchCV(random_f, parameters, scoring = f1_scorer, cv = 5, n_jobs = -1)
grid.fit(X_train, y_train)
model_est = grid.best_estimator_
model_params = grid.best_params_
grid.best_params_

{'class_weight': 'balanced', 'max_depth': 6}

In [226]:
# Random forest with my favourite state, state 42
rfc = RandomForestClassifier(random_state = 42, class_weight = 'balanced', max_depth = 6)
rfc.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=6, random_state=42)

In [227]:
# Storing in the results
recall_list.append(np.mean(cross_val_score(rfc,  X_train, y_train, scoring = 'recall', cv = 5)))
f1_list.append(np.mean(cross_val_score(rfc,  X_train, y_train, scoring = 'f1', cv = 5)))
precision_list.append(np.mean(cross_val_score(rfc,  X_train, y_train, scoring = 'precision', cv = 5)))
accuracy_list.append(np.mean(cross_val_score(rfc,  X_train, y_train, scoring = 'accuracy', cv = 5)))
auc_list.append(np.mean(cross_val_score(rfc, X_train, y_train, scoring = "roc_auc", cv = 5)))

In [228]:
# Metrics for testing set
recall_testing = []
f1_testing = []
precision_testing = []
acc_testing = []

In [229]:
# on the testing set and store results
y_pred = rfc.predict(X_test)
precision_testing.append(np.mean(metrics.precision_score(y_test, y_pred)))
recall_testing.append(np.mean(metrics.recall_score(y_test, y_pred)))
f1_testing.append(np.mean(metrics.f1_score(y_test, y_pred)))
acc_testing.append(np.mean(metrics.accuracy_score(y_test,y_pred)))
confusion_matrix(y_test, y_pred)

array([[20262, 18453],
       [ 6090, 15945]], dtype=int64)

In [230]:

print("\t \t The Model Metric Matrix for the training set")
# Cross validation.
metrics_summary = pd.DataFrame()
metrics_summary['Model #'] = [ 'Log Regression', 'Random Forest Model ']
metrics_summary['Recall'] = recall_list
metrics_summary['Accuracy'] = accuracy_list
metrics_summary['Precision'] = precision_list
metrics_summary['AUC'] = auc_list
metrics_summary['F1_score'] = f1_list
metrics_summary1 = metrics_summary.copy()
metrics_summary1

	 	 The Model Metric Matrix for the training set


,Model #,Recall,Accuracy,Precision,AUC,F1_score
0,Log Regression,0.341934,0.665808,0.564966,0.676664,0.426017
1,Random Forest Model,0.721820,0.599630,0.466460,0.672469,0.566689


In [231]:
sum(df['response_var'])/df['response_var'].count()

0.3627085610440233